In [1]:
from utils.dataloader import prepare_dataset
from config import config

print("Préparation du dataset...")
train_set, val_set, test_set = prepare_dataset()

print(f"\nNombre de locuteurs sélectionnés : {config['num_speakers']}")
print(f"Durée d'un segment audio : {config['segment_duration']} secondes")
print(f"Type de features : {config['feature_type']}")

print(f"\nTaille du set d'entraînement : {len(train_set)}")
print(f"Taille du set de validation   : {len(val_set)}")
print(f"Taille du set de test         : {len(test_set)}")

x, y = train_set[0]
print(f"\nShape d'un échantillon audio (x) : {x.shape}")
print(f"Label correspondant (y)         : {y}")

Préparation du dataset...
Loading of train-clean-100 subset...
Loading of train-clean-100 done.
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "d:\Speaker_recognition\venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\MATEBOOK D14\AppData\Local\Temp\ipykernel_15112\4054316915.py", line 5, in <module>
    train_set, val_set, test_set = prepare_dataset()
                                   ^^^^^^^^^^^^^^^^^
  File "d:\Speaker_recognition\utils\dataloader.py", line 86, in prepare_dataset
    for waveform, sample_rate, _, speaker_id, _, _ in dataset:
  File "d:\Speaker_recognition\venv\Lib\site-packages\torchaudio\datasets\librispeech.py", line 170, in __getitem__
    waveform = _load_waveform(self._archive, metadata[0], metadata[1])
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Speaker_recognition\venv\Lib\site-packages\torchaudio\datasets\utils.py", line 51, in _load_waveform
    waveform, sample_rate = torchaudio.load(path)
                            